In [40]:
from matplotlib import font_manager, rc  # 한글사용 matplotlib
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import csv
import codecs
import numpy as np
import datetime
import json
import tensorflow as tf
import jpype
from konlpy.tag import Okt
from IPython.core.interactiveshell import InteractiveShell
from pandas import DataFrame as df
InteractiveShell.ast_node_interactivity = 'all'
os.chdir("C:/Users/windd/pythondata")
%matplotlib inline
font_name = font_manager.FontProperties(
    fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

In [41]:
df_afsnt = pd.read_csv('bigcontest/2019빅콘테스트_퓨처스리그2/AFSNT.csv',engine='python',encoding='cp949')
df_afsnt["STT"] = pd.to_datetime(df_afsnt["STT"], format = "%H:%M").dt.time
df_afsnt["ATT"] = pd.to_datetime(df_afsnt["ATT"], format = "%H:%M").dt.time

df_afsnt = df_afsnt.rename(columns={ 
    "SDT_YY": "year", "SDT_MM":"month", "SDT_DD":"day",  "SDT_DY":"wday", 
    "ARP":"origin", "ODP":"dest", 
    "FLO":"airline", "FLT":"flight" ,"REG":"tailnum", 
    "AOD":"is_arrive", 
    "IRR":"is_regular", 
    "STT":"sched_time", "ATT":"real_time", 
    "DLY":"is_delay", "DRR":"cause_delay", 
    "CNL":"is_cancel", "CNR":"cause_cancel"
})
df_afsnt["date"] = pd.to_datetime(df_afsnt[["year", "month", "day"]])
df_afsnt["sched_datetime"] = pd.to_datetime(df_afsnt['date'].astype(str) + " " + df_afsnt["sched_time"].astype(str))
df_afsnt["real_datetime"] = pd.to_datetime(df_afsnt['date'].astype(str) + " " + df_afsnt["real_time"].astype(str))

# delay 변수 생성
temp = df_afsnt["real_datetime"] - df_afsnt["sched_datetime"] 
temp = pd.DataFrame(temp.dt.components)
df_afsnt["delay"] = temp['minutes'] + 60 * temp['hours'] + 1440 * temp['days']
del temp

df_afsnt.head()

,year,month,day,wday,origin,dest,airline,flight,tailnum,is_arrive,...,sched_time,real_time,is_delay,cause_delay,is_cancel,cause_cancel,date,sched_datetime,real_datetime,delay
0,2017,1,1,일,ARP3,ARP6,J,J1955,SEw3NzE4,D,...,10:05:00,10:32:00,N,NaN,N,NaN,2017-01-01,2017-01-01 10:05:00,2017-01-01 10:32:00,27
1,2017,1,1,일,ARP3,ARP6,J,J1954,SEw3NzE4,A,...,09:30:00,09:31:00,N,NaN,N,NaN,2017-01-01,2017-01-01 09:30:00,2017-01-01 09:31:00,1
2,2017,1,1,일,ARP3,ARP6,J,J1956,SEw3NzE4,A,...,12:45:00,13:03:00,N,NaN,N,NaN,2017-01-01,2017-01-01 12:45:00,2017-01-01 13:03:00,18
3,2017,1,1,일,ARP3,ARP6,J,J1957,SEw3NzE4,D,...,13:25:00,14:09:00,Y,C02,N,NaN,2017-01-01,2017-01-01 13:25:00,2017-01-01 14:09:00,44
4,2017,1,1,일,ARP3,ARP6,J,J1958,SEw3NzE4,A,...,16:10:00,16:31:00,N,NaN,N,NaN,2017-01-01,2017-01-01 16:10:00,2017-01-01 16:31:00,21


In [68]:
# 결항 데이터 제거
# tailnum 널값 제거 4개 정도임 결항 제거하면
# ARP1 -> ARP1 으로 가는 이상 데이터 제거
# 총 데이터는 979436개 

df_afsnt = df_afsnt[df_afsnt['is_cancel']=='N']
df_afsnt = df_afsnt[~(df_afsnt['tailnum'].isnull())]
df_afsnt = df_afsnt[~((df_afsnt['origin']=='ARP1') & (df_afsnt['dest']=='ARP1'))]
len(df_afsnt)

979436

In [69]:
df_afsnt.isnull().sum()

year                   0
month                  0
day                    0
wday                   0
origin                 0
dest                   0
airline                0
flight                 0
tailnum                0
is_arrive              0
is_regular             0
sched_time             0
real_time              0
is_delay               0
cause_delay       860504
is_cancel              0
cause_cancel      979436
date                   0
sched_datetime         0
real_datetime          0
delay                  0
dtype: int64

In [ ]:
# 파생변수 - 혼잡도

In [70]:
df_afsnt_d = df_afsnt[df_afsnt['is_arrive']=='D']
df_afsnt_a = df_afsnt[df_afsnt['is_arrive']=='A']

In [ ]:
# 연습용

test = df_afsnt_d.iloc[:1000]
test = test.sort_values(['origin','date','sched_time'])
test.index = list(range(len(test)))


base_time = datetime.timedelta(minutes=30)
congestion = []

for i in range(len(test)):
    
    airport = test['origin'][i]
    sched_time = test['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time + base_time
    congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(test)-30) else len(test) for x in [i+30]]
    
    
    for j in range(num[0], num2[0],1):
        if (airport == test['origin'][j]) and (lower_time <= test['sched_datetime'][j]) and (test['sched_datetime'][j] <= upper_time):
            congestion_cnt +=1
    congestion.append(congestion_cnt)

In [37]:
# 연습용

test = df_afsnt_a.iloc[:1000]
test = test.sort_values(['dest','date','sched_time'])
test.index = list(range(len(test)))


base_time = datetime.timedelta(minutes=30)
congestion = []

for i in range(len(test)):
    
    airport = test['dest'][i]
    sched_time = test['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time + base_time
    congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(test)-30) else len(test) for x in [i+30]]
    
    
    for j in range(num[0], num2[0],1):
        if (airport == test['dest'][j]) and (lower_time <= test['sched_datetime'][j]) and (test['sched_datetime'][j] <= upper_time):
            congestion_cnt +=1
    congestion.append(congestion_cnt)

In [39]:
test['congestion'] = congestion
test[['date','origin','dest','tailnum','sched_datetime','congestion']]

,date,origin,dest,tailnum,sched_datetime,congestion
0,2017-01-01,ARP3,ARP1,SEw3Nzg1,2017-01-01 07:50:00,2
1,2017-01-01,ARP12,ARP1,SEw3NTY4,2017-01-01 07:55:00,2
2,2017-01-01,ARP9,ARP1,SEw3NTY5,2017-01-01 08:35:00,2
3,2017-01-01,ARP2,ARP1,SEw3NzA2,2017-01-01 08:55:00,4
4,2017-01-01,ARP3,ARP1,SEw3NzU3,2017-01-01 09:10:00,3
5,2017-01-01,ARP11,ARP1,SEw3NTY2,2017-01-01 09:20:00,4
6,2017-01-01,ARP3,ARP1,SEw3NzI0,2017-01-01 09:50:00,5
7,2017-01-01,ARP3,ARP1,SEw3NzU4,2017-01-01 09:55:00,4
8,2017-01-01,ARP3,ARP1,SEw3NTI1,2017-01-01 10:10:00,4
9,2017-01-01,ARP3,ARP1,SEw3NzA4,2017-01-01 10:15:00,4


In [72]:
# 구하는 방식은 그대로 사용하면됨
# 다만 전처리하면서 제거되는 행들에 의해서 값이 변하기 때문에
# 나중에 다시 돌려야함
# 행의 출발예정 시간 기준으로 +- 30분 사이의 예정 비행기들을 카운드하여 혼잡도 계산

In [73]:
df_afsnt_d = df_afsnt_d.sort_values(['origin','date','sched_time'])
df_afsnt_d.index = list(range(len(df_afsnt_d)))

base_time = datetime.timedelta(minutes=30)
congestion_d = []

for i in range(len(df_afsnt_d)):
    
    airport = df_afsnt_d['origin'][i]
    sched_time = df_afsnt_d['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time + base_time
    congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(df_afsnt_d)-30) else len(df_afsnt_d) for x in [i+30]]
    
    for j in range(num[0], num2[0],1):
        if (airport == df_afsnt_d['origin'][j]) and (lower_time <= df_afsnt_d['sched_datetime'][j]) and (df_afsnt_d['sched_datetime'][j] <= upper_time):
            congestion_cnt +=1
    congestion_d.append(congestion_cnt)
    
df_afsnt_d['congestion'] = congestion_d

In [74]:
df_afsnt_a = df_afsnt_a.sort_values(['dest','date','sched_time'])
df_afsnt_a.index = list(range(len(df_afsnt_a)))

base_time = datetime.timedelta(minutes=30)
congestion_a = []

for i in range(len(df_afsnt_a)):
    
    airport = df_afsnt_a['dest'][i]
    sched_time = df_afsnt_a['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time + base_time
    congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(df_afsnt_a)-30) else len(df_afsnt_a) for x in [i+30]]
    
    for j in range(num[0], num2[0],1):
        if (airport == df_afsnt_a['dest'][j]) and (lower_time <= df_afsnt_a['sched_datetime'][j]) and (df_afsnt_a['sched_datetime'][j] <= upper_time):
            congestion_cnt +=1
    congestion_a.append(congestion_cnt)
    
df_afsnt_a['congestion'] = congestion_a

In [75]:
df_afsnt_d[['origin','dest','date','sched_time','congestion']].head(100)

,origin,dest,date,sched_time,congestion
0,ARP1,ARP3,2017-01-01,06:10:00,8
1,ARP1,ARP3,2017-01-01,06:15:00,10
2,ARP1,ARP3,2017-01-01,06:20:00,10
3,ARP1,ARP3,2017-01-01,06:25:00,10
4,ARP1,ARP3,2017-01-01,06:30:00,13
5,ARP1,ARP3,2017-01-01,06:30:00,13
6,ARP1,ARP3,2017-01-01,06:40:00,14
7,ARP1,ARP3,2017-01-01,06:40:00,14
8,ARP1,ARP3,2017-01-01,06:45:00,13
9,ARP1,ARP3,2017-01-01,06:45:00,13


In [93]:
df_afsnt_a[['origin','dest','date','sched_time','congestion']].head(100)

,origin,dest,date,sched_time,congestion
0,ARP3,ARP1,2017-01-01,07:15:00,7
1,ARP3,ARP1,2017-01-01,07:20:00,9
2,ARP3,ARP1,2017-01-01,07:25:00,11
3,ARP3,ARP1,2017-01-01,07:30:00,11
4,ARP3,ARP1,2017-01-01,07:35:00,12
5,ARP3,ARP1,2017-01-01,07:40:00,14
6,ARP3,ARP1,2017-01-01,07:45:00,14
7,ARP3,ARP1,2017-01-01,07:50:00,13
8,ARP3,ARP1,2017-01-01,07:50:00,13
9,ARP12,ARP1,2017-01-01,07:55:00,13


In [ ]:
# 파생변수 - 링크 데이터


In [77]:
df_afsnt_sorted_d = df_afsnt_d.sort_values(['date','tailnum','sched_datetime'])
df_afsnt_sorted_a = df_afsnt_a.sort_values(['date','tailnum','sched_datetime'])

In [78]:
df_afsnt_sorted_d.index = list(range(len(df_afsnt_sorted_d)))
len(df_afsnt_sorted_d)
len(df_afsnt_d)
len(df_afsnt_sorted_a)
len(df_afsnt_a)

489575

489575

489861

489861

In [79]:
df_afsnt_sorted_d.index = list(range(len(df_afsnt_sorted_d)))
link_ref_d = df_afsnt_sorted_d.pivot_table(index = ['date','tailnum'],aggfunc='count',values='is_arrive' )
link_ref_d.head(100)

link_ref_list_d = list(link_ref_d['is_arrive'].values)

is_arrive
date       tailnum            
2017-01-01 SEw3MjQ3          8
           SEw3NDYw          6
           SEw3NTI1          6
           SEw3NTI2          1
           SEw3NTQw          4
           SEw3NTU3          8
           SEw3NTU4          8
           SEw3NTY0          8
           SEw3NTY2          8
           SEw3NTY4          8
           SEw3NTY5          8
           SEw3NTYw          2
           SEw3NTYz          9
           SEw3NTk0          9
           SEw3NTk5          4
           SEw3NzA0          1
           SEw3NzA1          4
           SEw3NzA2          7
           SEw3NzA4          7
           SEw3NzAz         10
           SEw3NzE2          8
           SEw3NzE4          8
           SEw3NzE5          8
           SEw3NzEz          2
           SEw3NzI0          4
           SEw3NzI1          4
           SEw3NzI2          1
           SEw3NzI3          7
           SEw3NzIy          2
           SEw3NzIz          7
...                        ...
           SEw4MjA1          8
           SEw4MjA3          8
           SEw4MjEz          6
           SEw4MjI0          2
           SEw4MjIx          1
           SEw4MjM1          4
           SEw4MjM2          8
           SEw4MjM3         10
           SEw4MjM5          5
           SEw4MjQ0          1
           SEw4MjQ2          1
           SEw4MjQ3          4
           SEw4MjQw          1
           SEw4MjQx          6
           SEw4MjU2          2
           SEw4MjUz          8
           SEw4MjY0          8
           SEw4MjY4          7
           SEw4MjYx          6
           SEw4Mjc3          1
           SEw4Mjc4          1
           SEw4Mjg5         10
           SEw4Mjk3          9
           SEw4Mjky          4
2017-01-02 SEw3MjQ3          8
           SEw3NDYw          1
           SEw3NTI1          6
           SEw3NTU3          8
           SEw3NTU4          6
           SEw3NTY0          8

[100 rows x 1 columns]

In [80]:
#linkdata - departure

link_d = []

for i in link_ref_list_d:
    link_range = list(range(i))
    link_d.append(link_range)
    
link_d = sum(link_d, [])
df_afsnt_sorted_d['link'] = link_d

In [81]:
df_afsnt_sorted_a.index = list(range(len(df_afsnt_sorted_a)))
len(df_afsnt_sorted_a)

link_ref_a = df_afsnt_sorted_a.pivot_table(index = ['date','tailnum'],aggfunc='count',values='is_arrive' )
link_ref_a.head(100)

link_ref_list_a = list(link_ref_a['is_arrive'].values)

489861

is_arrive
date       tailnum            
2017-01-01 SEw3MjQ3          8
           SEw3NDYw          6
           SEw3NTI1          6
           SEw3NTQw          4
           SEw3NTU3          8
           SEw3NTU4          8
           SEw3NTY0          8
           SEw3NTY2          8
           SEw3NTY4          8
           SEw3NTY5          8
           SEw3NTYw          2
           SEw3NTYz          9
           SEw3NTk0          9
           SEw3NTk5          4
           SEw3NzA0          1
           SEw3NzA1          4
           SEw3NzA2          7
           SEw3NzA4          7
           SEw3NzAz         10
           SEw3NzE2          8
           SEw3NzE4          8
           SEw3NzE5          8
           SEw3NzEz          2
           SEw3NzI0          4
           SEw3NzI1          4
           SEw3NzI2          1
           SEw3NzI3          7
           SEw3NzIy          2
           SEw3NzIz          7
           SEw3NzM1          2
...                        ...
           SEw4MjA3          8
           SEw4MjEz          6
           SEw4MjI0          2
           SEw4MjIx          1
           SEw4MjM1          4
           SEw4MjM2          8
           SEw4MjM3         10
           SEw4MjM5          5
           SEw4MjQ0          1
           SEw4MjQ2          1
           SEw4MjQ3          4
           SEw4MjQw          1
           SEw4MjQx          6
           SEw4MjU2          2
           SEw4MjUz          8
           SEw4MjY0          8
           SEw4MjY4          7
           SEw4MjYx          6
           SEw4Mjc3          1
           SEw4Mjg5         10
           SEw4Mjk3          9
           SEw4Mjky          5
2017-01-02 SEw3MjQ3          8
           SEw3NTI1          6
           SEw3NTU3          8
           SEw3NTU4          6
           SEw3NTY0          8
           SEw3NTY2          4
           SEw3NTY4          8
           SEw3NTY5          8

[100 rows x 1 columns]

In [82]:
#linkdata - arrival

link_a = []

for i in link_ref_list_a:
    link_range = list(range(i))
    link_a.append(link_range)
    
link_a = sum(link_a, [])
df_afsnt_sorted_a['link'] = link_a

In [83]:
# 출발링크

df_afsnt_sorted_d[['date','origin','dest','tailnum','sched_datetime','link','is_arrive']].head(100)

,date,origin,dest,tailnum,sched_datetime,link,is_arrive
0,2017-01-01,ARP1,ARP3,SEw3MjQ3,2017-01-01 07:20:00,0,D
1,2017-01-01,ARP3,ARP1,SEw3MjQ3,2017-01-01 09:15:00,1,D
2,2017-01-01,ARP1,ARP3,SEw3MjQ3,2017-01-01 11:05:00,2,D
3,2017-01-01,ARP3,ARP1,SEw3MjQ3,2017-01-01 13:00:00,3,D
4,2017-01-01,ARP1,ARP3,SEw3MjQ3,2017-01-01 14:50:00,4,D
5,2017-01-01,ARP3,ARP1,SEw3MjQ3,2017-01-01 16:45:00,5,D
6,2017-01-01,ARP1,ARP3,SEw3MjQ3,2017-01-01 18:35:00,6,D
7,2017-01-01,ARP3,ARP1,SEw3MjQ3,2017-01-01 20:40:00,7,D
8,2017-01-01,ARP1,ARP3,SEw3NDYw,2017-01-01 09:55:00,0,D
9,2017-01-01,ARP3,ARP1,SEw3NDYw,2017-01-01 12:10:00,1,D


In [84]:
# 도착 링크

df_afsnt_sorted_a[['date','origin','dest','tailnum','sched_datetime','link','is_arrive']].head(100)

,date,origin,dest,tailnum,sched_datetime,link,is_arrive
0,2017-01-01,ARP3,ARP1,SEw3MjQ3,2017-01-01 08:30:00,0,A
1,2017-01-01,ARP1,ARP3,SEw3MjQ3,2017-01-01 10:25:00,1,A
2,2017-01-01,ARP3,ARP1,SEw3MjQ3,2017-01-01 12:15:00,2,A
3,2017-01-01,ARP1,ARP3,SEw3MjQ3,2017-01-01 14:10:00,3,A
4,2017-01-01,ARP3,ARP1,SEw3MjQ3,2017-01-01 16:00:00,4,A
5,2017-01-01,ARP1,ARP3,SEw3MjQ3,2017-01-01 17:55:00,5,A
6,2017-01-01,ARP3,ARP1,SEw3MjQ3,2017-01-01 19:45:00,6,A
7,2017-01-01,ARP1,ARP3,SEw3MjQ3,2017-01-01 21:50:00,7,A
8,2017-01-01,ARP3,ARP1,SEw3NDYw,2017-01-01 11:05:00,0,A
9,2017-01-01,ARP1,ARP3,SEw3NDYw,2017-01-01 13:20:00,1,A


In [85]:
# 공항 출발 time sequence 생성

df_afsnt_sorted_d2 = df_afsnt_sorted_d.sort_values(['date','origin','sched_datetime','real_datetime'])
df_afsnt_sorted_d2.index = list(range(len(df_afsnt_sorted_d2)))

# 공장 도착 time sequence 생성

df_afsnt_sorted_a2 = df_afsnt_sorted_a.sort_values(['date','dest','sched_datetime','real_datetime'])
df_afsnt_sorted_a2.index = list(range(len(df_afsnt_sorted_a2)))

In [86]:
# 출발

sequence_ref_d = df_afsnt_sorted_d2.pivot_table(index = ['date','origin'], aggfunc = 'count', values = 'is_arrive')
sequence_ref_list_d = list(sequence_ref_d['is_arrive'].values)

In [87]:
# 도착

sequence_ref_a = df_afsnt_sorted_a2.pivot_table(index = ['date','dest'], aggfunc = 'count', values = 'is_arrive')
sequence_ref_list_a = list(sequence_ref_a['is_arrive'].values)

In [88]:
# 출발 공항 sequence

sequence_d = []

for i in sequence_ref_list_d:
    sequence_range = list(range(i))
    sequence_d.append(sequence_range)
    
sequence_d = sum(sequence_d, [])
df_afsnt_sorted_d2['air_sequence'] = sequence_d

In [89]:
# 도착 공항 sequence

sequence_a = []

for i in sequence_ref_list_a:
    sequence_range = list(range(i))
    sequence_a.append(sequence_range)
    
sequence_a = sum(sequence_a, [])
df_afsnt_sorted_a2['air_sequence'] = sequence_a

In [90]:
df_afsnt_sorted_d2[['date','origin','dest','sched_datetime','tailnum','is_arrive','air_sequence']].head(100)

,date,origin,dest,sched_datetime,tailnum,is_arrive,air_sequence
0,2017-01-01,ARP1,ARP3,2017-01-01 06:10:00,SEw3Nzc2,D,0
1,2017-01-01,ARP1,ARP3,2017-01-01 06:15:00,SEw4MjM2,D,1
2,2017-01-01,ARP1,ARP3,2017-01-01 06:20:00,SEw4MjM3,D,2
3,2017-01-01,ARP1,ARP3,2017-01-01 06:25:00,SEw4MjA3,D,3
4,2017-01-01,ARP1,ARP3,2017-01-01 06:30:00,SEw4MDMx,D,4
5,2017-01-01,ARP1,ARP3,2017-01-01 06:30:00,SEw3NTYz,D,5
6,2017-01-01,ARP1,ARP3,2017-01-01 06:40:00,SEw4MDE1,D,6
7,2017-01-01,ARP1,ARP3,2017-01-01 06:40:00,SEw3NTk0,D,7
8,2017-01-01,ARP1,ARP3,2017-01-01 06:45:00,SEw3Nzcz,D,8
9,2017-01-01,ARP1,ARP3,2017-01-01 06:45:00,SEw3Nzg1,D,9


In [91]:
df_afsnt_sorted_a2[['date','origin','dest','sched_datetime','tailnum','is_arrive','air_sequence']].head(100)

,date,origin,dest,sched_datetime,tailnum,is_arrive,air_sequence
0,2017-01-01,ARP3,ARP1,2017-01-01 07:15:00,SEw3Nzc2,A,0
1,2017-01-01,ARP3,ARP1,2017-01-01 07:20:00,SEw4MjM2,A,1
2,2017-01-01,ARP3,ARP1,2017-01-01 07:25:00,SEw4MjM3,A,2
3,2017-01-01,ARP3,ARP1,2017-01-01 07:30:00,SEw4MjA3,A,3
4,2017-01-01,ARP3,ARP1,2017-01-01 07:35:00,SEw3NTYz,A,4
5,2017-01-01,ARP3,ARP1,2017-01-01 07:40:00,SEw4MDMx,A,5
6,2017-01-01,ARP3,ARP1,2017-01-01 07:45:00,SEw4MDE1,A,6
7,2017-01-01,ARP3,ARP1,2017-01-01 07:50:00,SEw3Nzg1,A,7
8,2017-01-01,ARP3,ARP1,2017-01-01 07:50:00,SEw3NTk0,A,8
9,2017-01-01,ARP3,ARP1,2017-01-01 07:55:00,SEw3Nzcz,A,9


In [94]:
len(df_afsnt)
len(df_afsnt_sorted_d2) +len(df_afsnt_sorted_a2)

979436

979436

In [96]:
df_afsnt_added = pd.concat([df_afsnt_sorted_d2, df_afsnt_sorted_a2],sort=False)

In [98]:
df_afsnt_added.to_excel('bigcontest/링크_시퀀스_혼잡도.xlsx')